# Naive Bayes - Análise de Sentimento

Para essa atividade vamos gerar uma modelo de análise de sentimento em inglês baseado em reviews retirados de 3 sites: Amazon, IMDb e Yelp. Essa base está disponível [neste link](https://archive.ics.uci.edu/ml/datasets/Sentiment+Labelled+Sentences). Mais detalhes podem ser encontrados no link ou no artigo de referência: *From Group to Individual Labels using Deep Features', Kotzias et. al,. KDD 2015*. 

A base possui um texto e para cada texto um sentimento sobre o conteúdo abordado no texto. Os sentimentos podem ser positivos (1) ou negativos (2). Foram coletados em média 500 textos para cada sentimento em cada base. 

A atividade consiste em construir uma modelo de aprendizagem para análise de sentimento em inglês utilizando o Naive Bayes. O primeiro passo foi carregar o Dataset de forma apropriada e em seguida construir a matriz de entrada para nosso algoritmo. As etapas do exercício juntamente com o que deve ser feito está descrito a seguir. 

## Carregando o Dataset

In [26]:
import pandas as pd

df_amazon = pd.read_csv("../datasets/sentimentanalysis/amazon_cells_labelled.txt", 
                        sep="\t", header=None, names=['Text','Sentiment'])
df_imdb = pd.read_csv("../datasets/sentimentanalysis/imdb_labelled.txt", 
                        sep="\t", header=None, names=['Text','Sentiment'])
df_yelp = pd.read_csv("../datasets/sentimentanalysis/yelp_labelled.txt", 
                        sep="\t", header=None, names=['Text','Sentiment'])

In [27]:
print("Amazon dataset %s" % str(df_amazon.shape))
print("IMDb dataset %s" % str(df_imdb.shape))
print("Yelp dataset %s" % str(df_yelp.shape))

Amazon dataset (1000, 2)
IMDb dataset (748, 2)
Yelp dataset (1000, 2)


In [28]:
join_frames = [df_amazon, df_imdb, df_yelp]

df_final_dataset = pd.concat(join_frames)

df_final_dataset.shape

(2748, 2)

## Construindo a base de dados

A base de dados possui 2748 textos que foram classificados em dois sentimentos: negativo (0) e positivo (1). Construa uma base de dados apropriada para os testes. Divida a base em treino e teste (80% para treino e 20% para teste). A base de treinamento será utilizado para a construção do modelo e a de teste para o teste final do modelo construído. 

In [29]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df_final_dataset['Text'], 
                                                    df_final_dataset['Sentiment'], 
                                                    random_state=1,
                                                    test_size=0.2
                                                   )

In [30]:
print('Number of rows in the total set: {}'.format(df_final_dataset.shape[0]))
print('Number of rows in the training set: {}'.format(X_train.shape[0]))
print('Number of rows in the test set: {}'.format(X_test.shape[0]))

Number of rows in the total set: 2748
Number of rows in the training set: 2198
Number of rows in the test set: 550


## Construindo o Bag of Words

Construa o Bag of Words para a base de treinamento. Para isso, utilize o método CountVectorizer como mostrado a seguir.

In [78]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
count_vector = CountVectorizer()
count_vector

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

O `CountVectorizer` permite construir o array que serve de entrada para os modelos de aprendizagem. O código a seguir, visualiza o array. 

In [32]:
training_data = count_vector.fit_transform(X_train)
training_data.shape

(2198, 4581)

Foi gerada uma matriz de 2198 linhas (os textos) e 4581 colunas (as palavras). Devemos fazer o mesmo com a base de teste. 

In [33]:
testing_data = count_vector.transform(X_test)
testing_data.shape

(550, 4581)

Foi gerada uma matriz com 550 linhas e 4581 colunas também. `training_data` e `testing_data` são as estruturas que devem ser utilizadas no modelo Naive Bayes.

## Atividade 1

Implemente um modelo Naive Bayes para a base gerada. Utilize validação cruzada de 5 folds na base de treinamento e em seguida teste o modelo gerado na base de testes. Reporte a acurácia resultante da validação cruzada e da base de testes. Teste os 3 tipos de Naive Bayes presentes no `scikit-learn`. 

In [34]:
from sklearn.naive_bayes import MultinomialNB, GaussianNB, BernoulliNB
import pandas as pd
import numpy as np

from sklearn.model_selection import cross_val_score

naive1 = MultinomialNB()
naive2 = GaussianNB()
naive3 = BernoulliNB()

algorithms = [naive1, naive2, naive3]

for algo in algorithms:
    scores = cross_val_score(algo, training_data.toarray(), y_train, cv=5, scoring='accuracy')
    score_mean = scores.mean()
    print("Acc do %s (treino): %f " % (algo.__class__.__name__, score_mean))
    
    algo.fit(training_data.toarray(), y_train)
    score_test = algo.score(testing_data.toarray(), y_test)
    print("Acc do %s (teste): %f " % (algo.__class__.__name__, score_test))
    print()

Acc do MultinomialNB (treino): 0.813474 
Acc do MultinomialNB (teste): 0.812727 

Acc do GaussianNB (treino): 0.653311 
Acc do GaussianNB (teste): 0.645455 

Acc do BernoulliNB (treino): 0.802111 
Acc do BernoulliNB (teste): 0.800000 



Sendo assim, o MultinomialNB foi o melhor modelo utilizando validação cruzada de 5 folds. 

### Atividade 2

O Naive Bayes em si não tem muitos parâmetros para ajustar. No entanto, podemos ajustar no pré-processamento. Quando utilizamos a classe `CountVectorizer` podemos utilizar uma série de técnicas de pré-processamento para melhorar os dados de entrada do modelo. 

Pesquise sobre o `CountVectorizer` e modifique os parâmetros `default` para gerar dados melhores e, consequentemente, um modelo melhor do que o construído na Atividade 1. Reporte seus resultados na validação cruzada e nos testes. Utilize somente o tipo de Naive Bayes que teve melhor resultado na atividade 1. 

In [234]:
X_train, X_test, y_train, y_test = train_test_split(df_final_dataset['Text'], 
                                                    df_final_dataset['Sentiment'], 
                                                    random_state=1,
                                                    test_size=0.3
                                                   )

In [235]:
print('Number of rows in the total set: {}'.format(df_final_dataset.shape[0]))
print('Number of rows in the training set: {}'.format(X_train.shape[0]))
print('Number of rows in the test set: {}'.format(X_test.shape[0]))

Number of rows in the total set: 2748
Number of rows in the training set: 1923
Number of rows in the test set: 825


Para essa atividade, precisamos gerar novamente nossa matriz de entrada para os modelos. Como pedido na questão, o método `CountVectorizer` permite transformar os textos nessa tabela. No entanto, no exemplo apresentado anteriormente não mudamos nenhuma das informações padrões. Vamos mudar algumas coisas e tentar chegar em um modelo melhor que o construído. 

Nosso melhor exemplo foi obtido utilizando o `MultinomialNB` com acurácia de `0.813474`. Vamos trabalhar somente com esse modelo para essa questão. 

### CountVectorizer

Informações sobre o `CountVectorizer` podem ser encontrados [neste link](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html) da documentação do `scikit-learn`.

Como já foi dito anteriormente, na classificação de textos nossos atributos são formados pelas palavras que compõe o conjunto de textos trabalhados. No exemplo trabalhado, essas colunas consistem em todas as palavras. No entanto, nem sempre precisamos de todas palavras. É fácil perceber que existem palavras que não fornencem nenhuma informação para a construção do modelo de classificação. 

Sendo assim, nossa primeira alteração no modelo é limitar o número de atributos através do atributo `max_features`. Vamos ver o que isso implica na acurácia. 


### Modelo Original sem alterações

In [236]:
count_vector = CountVectorizer()
training_data = count_vector.fit_transform(X_train)
testing_data = count_vector.transform(X_test)

print("Tamanho do Vocabulário %i " % (len(count_vector.vocabulary_.keys())))

algo_final = MultinomialNB()

scores = cross_val_score(algo_final, training_data.toarray(), y_train, cv=5, scoring='accuracy')
print("Acc do %s (treino): %f " % (algo_final.__class__.__name__, score_mean))

algo_final.fit(training_data.toarray(), y_train)
score_test = algo_final.score(testing_data.toarray(), y_test)
print("Acc do %s (teste) %f " % (algo_final.__class__.__name__, score_test))

Tamanho do Vocabulário 4265 
Acc do MultinomialNB (treino): 0.802111 
Acc do MultinomialNB (teste) 0.787879 


### Modelo com as modificações

In [88]:
from sklearn.feature_extraction import text 

In [248]:
count_vector = CountVectorizer(max_features=300, 
                               ngram_range=(1,2), 
                               stop_words=text.ENGLISH_STOP_WORDS,
                               analyzer='word',
                               max_df=0.8,
                               min_df=5)

training_data = count_vector.fit_transform(X_train)
testing_data = count_vector.transform(X_test)

print("Tamanho do Vocabulário %i " % (len(count_vector.vocabulary_.keys())))

algo_final = MultinomialNB()

scores = cross_val_score(algo_final, training_data.toarray(), y_train, cv=5, scoring='accuracy')
print("Acc do %s (treino): %f " % (algo_final.__class__.__name__, score_mean))

algo_final.fit(training_data.toarray(), y_train)
score_test = algo_final.score(testing_data.toarray(), y_test)
print("Acc do %s (teste) %f " % (algo_final.__class__.__name__, score_test))

Tamanho do Vocabulário 300 
Acc do MultinomialNB (treino): 0.802111 
Acc do MultinomialNB (teste) 0.744242 


In [250]:
array_count = np.asarray(training_data.sum(axis=0))[0]
features = count_vector.get_feature_names()

for e in  zip(features, array_count):
    print("%s : %i" % (e[0], e[1]))

10 : 25
20 : 8
absolutely : 13
acting : 30
actors : 13
actually : 12
amazing : 27
area : 8
art : 12
atmosphere : 10
avoid : 15
away : 9
awesome : 13
awful : 18
bad : 70
battery : 32
battery life : 9
beautiful : 10
believe : 7
best : 58
better : 44
big : 15
bit : 13
black : 15
bland : 11
bluetooth : 16
boring : 9
bought : 15
breakfast : 9
burger : 10
buy : 16
calls : 10
came : 22
camera : 13
car : 15
care : 9
case : 28
cast : 14
cell : 11
certainly : 10
character : 17
characters : 27
charge : 12
charger : 13
cheap : 15
chicken : 15
clear : 10
cold : 8
come : 10
comfortable : 13
coming : 12
completely : 11
cool : 14
couldn : 12
crap : 9
customer : 12
customer service : 12
day : 13
deal : 11
definitely : 24
delicious : 10
design : 10
device : 9
dialogue : 11
did : 33
didn : 32
different : 11
director : 10
disappointed : 28
disappointing : 9
disappointment : 9
does : 27
doesn : 22
don : 48
don waste : 10
dropped : 8
ear : 20
easy : 14
eat : 11
effects : 9
end : 13
ending : 10
enjoy : 8
enj